In [4]:
import json
import requests
DSVM_TEMPLATE_URL = "https://raw.githubusercontent.com/Azure/DataScienceVM/master/Scripts/CreateDSVM/Ubuntu/azuredeploy.json"

In [77]:
template = requests.get(DSVM_TEMPLATE_URL).json()

In [78]:
template

{'$schema': 'https://schema.management.azure.com/schemas/2015-01-01/deploymentTemplate.json#',
 'contentVersion': '1.0.0.0',
 'parameters': {'adminUsername': {'type': 'string',
   'metadata': {'description': 'Username for the Virtual Machine.'}},
  'adminPassword': {'type': 'securestring',
   'metadata': {'description': 'Password for the Virtual Machine.'}},
  'vmName': {'type': 'string',
   'metadata': {'description': 'Name Prefix for the Virtual Machine.'}},
  'vmSize': {'type': 'string',
   'defaultValue': 'Standard_DS3_v2',
   'metadata': {'description': 'Size for the Virtual Machine.'}}},
 'variables': {'location': '[resourceGroup().location]',
  'imagePublisher': 'microsoft-dsvm',
  'imageOffer': 'linux-data-science-vm-ubuntu',
  'OSDiskName': "[concat(parameters('vmName'),'-osdisk')]",
  'DataDiskName': "[concat(parameters('vmName'),'-data-0')]",
  'sku': 'linuxdsvmubuntu',
  'nicName': "[parameters('vmName')]",
  'addressPrefix': '10.0.0.0/16',
  'subnetName': 'Subnet',
  'subn

In [79]:
vm_props = [i["properties"] for i in template["resources"] if i["type"] == "Microsoft.Compute/virtualMachines"][0]

In [80]:
vm_stor_profile = vm_props["storageProfile"]

In [81]:
vm_stor_profile["osDisk"]["diskSizeGB"] = 100
vm_stor_profile["osDisk"]["managedDisk"]["storageAccountType"] = "Premium_LRS"

In [82]:
vm_stor_profile["dataDisks"][0]["diskSizeGB"] = 200
vm_stor_profile["dataDisks"][0]["managedDisk"]["storageAccountType"] = "Premium_LRS"

In [92]:
vm_os_profile = vm_props["osProfile"]

In [93]:
vm_os_profile["linuxConfiguration"] = {"ssh": {
                            "publicKeys": [
                            {
                                "path": "[variables('sshKeyPath')]",
                                "keyData": "[parameters('sshKeyData')]"
                            }
                            ]
                        }}

In [98]:
template["parameters"]["sshKeyData"] =  {
            "type": "string",
          "metadata": {
              "description": "SSH public key data as a string."
          }}

template["variables"]["sshKeyPath"] =  "[concat('/home/',parameters('adminUsername'),'/.ssh/authorized_keys')]"


In [94]:
with open("/Users/vykhand/.ssh/id_rsa.pub", "r") as f:
    ssh_key = f.read().replace("\n", "")

In [83]:
from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.resource.resources.models import DeploymentMode

In [84]:
subscription_id = "81ae6a7a-0699-4b60-9c61-294bae201fba"
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
client_id = "e14ed072-80aa-4782-881d-5362589e9d32"
client_secret = "A/PeHoOuHmx0TieuDzGEhuwR0s5gXTTRS7RBvuq82zY="

In [85]:
credentials  = ServicePrincipalCredentials(client_id, client_secret, tenant = tenant_id)

In [86]:
client = ResourceManagementClient(credentials, subscription_id)

In [87]:
location = "westeurope"

group_name = "dsvm001"
vm_name = "dsvm001"
vm_size = "Standard_D4s_v3"
admin_name = "vykhand"
admin_pass = "P00l!2018"

In [166]:
op = client.resource_groups.create_or_update(group_name, {"location": location})

In [167]:
parameters = {
    'sshKeyData': ssh_key,
    'vmName': vm_name,
    'vmSize': vm_size,
    'adminUsername': admin_name,
    'adminPassword': admin_pass
}

parameters = {k: {'value': v} for k, v in parameters.items()}

deployment_properties = {
            'mode': DeploymentMode.incremental,
            'template': template,
            'parameters': parameters
        }

In [168]:
deployment_async_operation = client.deployments.create_or_update(group_name, 'deployment-' + vm_name, deployment_properties)
deployment_async_operation.wait()

In [123]:
from azure.mgmt.compute import ComputeManagementClient

In [139]:
from azure.mgmt.network import NetworkManagementClient

In [125]:
compute_client  = ComputeManagementClient(credentials, subscription_id)

In [140]:
network_client = NetworkManagementClient(credentials, subscription_id)

In [146]:
vm.network_profile.network_interfaces[0].id

'/subscriptions/81ae6a7a-0699-4b60-9c61-294bae201fba/resourceGroups/dsvm001/providers/Microsoft.Network/networkInterfaces/dsvm001'

In [126]:
vm = compute_client.virtual_machines.get(group_name, vm_name)

In [138]:
vm.as_dict()

{'id': '/subscriptions/81ae6a7a-0699-4b60-9c61-294bae201fba/resourceGroups/dsvm001/providers/Microsoft.Compute/virtualMachines/dsvm001',
 'name': 'dsvm001',
 'type': 'Microsoft.Compute/virtualMachines',
 'location': 'westeurope',
 'tags': {'Application': 'DataScience'},
 'hardware_profile': {'vm_size': 'Standard_D4s_v3'},
 'storage_profile': {'image_reference': {'publisher': 'microsoft-dsvm',
   'offer': 'linux-data-science-vm-ubuntu',
   'sku': 'linuxdsvmubuntu',
   'version': 'latest'},
  'os_disk': {'os_type': 'Linux',
   'name': 'dsvm001-osdisk',
   'caching': 'ReadWrite',
   'create_option': 'FromImage',
   'disk_size_gb': 100,
   'managed_disk': {'id': '/subscriptions/81ae6a7a-0699-4b60-9c61-294bae201fba/resourceGroups/dsvm001/providers/Microsoft.Compute/disks/dsvm001-osdisk',
    'storage_account_type': 'Premium_LRS'}},
  'data_disks': [{'lun': 0,
    'name': 'dsvm001-data-0',
    'caching': 'None',
    'create_option': 'FromImage',
    'disk_size_gb': 200,
    'managed_disk': {

In [109]:
vm_host = "dsvm001.westeurope.cloudapp.azure.com"

In [174]:
op = compute_client.virtual_machines.run_command(group_name, vm_name, 
                                                 parameters = {'command_id': 'RunShellScript',
                                                              "script": ["sudo growpart /dev/sdc 1",
                                                                        "sudo resize2fs /dev/sdc1"]})
op.wait()

In [103]:
import paramiko

In [115]:
ssh = paramiko.SSHClient()
ssh.load_system_host_keys()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

In [116]:
ssh.connect(vm_host,username = "vykhand",  key_filename="/Users/vykhand/.ssh/id_rsa" )

In [117]:
ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command("sudo growpart /dev/sdc 1")

In [118]:
ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command("sudo resize2fs /dev/sdc1")

In [152]:
compute_client.virtual_machines.deallocate(group_name, vm_name)

In [175]:
op = client.resource_groups.delete(group_name)

In [176]:
op.wait()

In [158]:
compute_client.virtual_machines.list(group_name)

In [160]:
[g for g in client.resource_groups.list() if g.name == group_name]

[]

In [163]:
[v for v in compute_client.virtual_machines.list_all()]

[]